In [1]:
from google.cloud import bigquery
bq = bigquery.Client()

In [2]:
q = bq.query(
"""
select count(*) as num_rows 
from bigquery-public-data.geo_us_boundaries.counties
""")
q.to_dataframe()

,num_rows
0,3233


In [3]:
#q1
q1 = bq.query(
"""
select (area_water_meters/(area_land_meters+area_water_meters))*100 as percentage_covered_by_water
from bigquery-public-data.geo_us_boundaries.counties
where county_name = 'Dane'
""")
result = q1.to_dataframe()
result['percentage_covered_by_water'][0]

3.314489915768649

In [4]:
#q2
q2 = bq.query(
"""
select s.state, count(c.state_fips_code) as num_counties
from bigquery-public-data.geo_us_boundaries.counties c
inner join bigquery-public-data.geo_us_boundaries.states s on c.state_fips_code = s.state_fips_code
group by s.state
order by num_counties DESC
limit 5
""")
q2.to_dataframe().set_index('state')['num_counties'].to_dict()

{'TX': 254, 'GA': 159, 'VA': 133, 'KY': 120, 'MO': 115}

In [5]:
no_cache = bigquery.QueryJobConfig(use_query_cache=False)
q1_cost = bq.query(
"""
select (area_water_meters/area_land_meters)*100 as percentage_covered_by_water
from bigquery-public-data.geo_us_boundaries.counties
where county_name = 'Dane'
""", job_config=no_cache)
q2_cost = bq.query(
"""
select s.state, count(c.state_fips_code) as num_counties
from bigquery-public-data.geo_us_boundaries.counties c
inner join bigquery-public-data.geo_us_boundaries.states s on c.state_fips_code = s.state_fips_code
group by s.state
order by num_counties DESC
limit 5
""", job_config=no_cache)

In [6]:
#q3
tokyo_cost = 6.25 # per TB
q1_cost_q = q1_cost.total_bytes_billed / 1024**4 * tokyo_cost
q2_cost_q = q2_cost.total_bytes_billed / 1024**4 * tokyo_cost
{'q1': q1_cost_q, 'q2': q2_cost_q}

{'q1': 5.9604644775390625e-05, 'q2': 0.00011920928955078125}

In [7]:
source = "gs://cs544ymp8bucket/hdma-wi-2021-split.parquet"
project = "united-park-412420" 
dataset = "p8"
ds = bigquery.Dataset(f"{project}.{dataset}")
bq.create_dataset(ds, exists_ok=True)
job_config = bigquery.LoadJobConfig(source_format="PARQUET", write_disposition="WRITE_TRUNCATE")
job = bq.load_table_from_uri(source, f"{project}.{dataset}.hdma", job_config=job_config)
job.result()

LoadJob<project=united-park-412420, location=US, id=2f85fdcf-5860-49a8-975f-7c22c7395fc2>

In [8]:
#q4
[ds.dataset_id for ds in bq.list_datasets("united-park-412420")]

['p8']

In [9]:
#q5
q5 = bq.query(
"""
select c.county_name, count(h.county_code) as loan_count
from p8.hdma h
join bigquery-public-data.geo_us_boundaries.counties c on h.county_code = c.county_fips_code
group by c.county_name
order by loan_count DESC
limit 10;
""")
q5.to_dataframe().set_index('county_name')['loan_count'].to_dict()

{'Milwaukee': 46570,
 'Dane': 38557,
 'Waukesha': 34159,
 'Brown': 15615,
 'Racine': 13007,
 'Outagamie': 11523,
 'Kenosha': 10744,
 'Washington': 10726,
 'Rock': 9834,
 'Winnebago': 9310}

In [10]:
url = "https://docs.google.com/spreadsheets/d/1FfalqAWdzz01D1zIvBxsDWLW05-lvANWjjAj2vI4A04/"

external_config = bigquery.ExternalConfig("GOOGLE_SHEETS")
external_config.source_uris = [url]
external_config.options.skip_leading_rows = 1
external_config.autodetect = True

table = bigquery.Table("united-park-412420.p8.applications")
table.external_data_configuration = external_config

table = bq.create_table(table, exists_ok=True)
table

Table(TableReference(DatasetReference('united-park-412420', 'p8'), 'applications'))

In [11]:
#q6
q6 = bq.query("""
SELECT COUNT(*) as application_count
FROM united-park-412420.p8.applications
WHERE income = 200000
""")
application_count = q6.to_dataframe().iloc[0]['application_count']
application_count

9

In [12]:
#q7
q7 = bq.query("""
SELECT c.county_name, COUNT(*) as application_count
FROM united-park-412420.p8.applications as apps
JOIN `bigquery-public-data.geo_us_boundaries.counties` as c
ON ST_WITHIN(ST_GEOGPOINT(apps.longitude, apps.latitude), c.county_geom)
WHERE c.state_fips_code = '55'
GROUP BY c.county_name
""")
q7.to_dataframe().set_index('county_name')['application_count'].to_dict()

{'Dane': 16,
 'Door': 6,
 'Milwaukee': 3,
 'Marinette': 3,
 'Walworth': 5,
 'Fond du Lac': 1,
 'Outagamie': 1,
 'Sauk': 1,
 'Oneida': 3,
 'Douglas': 1,
 'Columbia': 1,
 'Green Lake': 1,
 'Jefferson': 2,
 'Brown': 2,
 'Kewaunee': 1,
 'Monroe': 2,
 'Florence': 1,
 'Bayfield': 1,
 'Barron': 1,
 'Juneau': 1,
 'Sheboygan': 1,
 'Winnebago': 1}

In [13]:
%load_ext google.cloud.bigquery

In [14]:
%%bigquery
CREATE OR REPLACE MODEL `united-park-412420.p8.model`
OPTIONS(model_type="LINEAR_REG", input_label_cols=["loan_amount"]) 

AS

SELECT income, loan_term, loan_amount
FROM
  p8.hdma
WHERE
  dataset = 'train'


Query is running:   0%|          |

""


In [31]:
#q8

q = bq.query(
"""
SELECT mean_absolute_error
FROM ML.EVALUATE(MODEL p8.model, (
  SELECT income, loan_term, loan_amount
  FROM `p8.hdma`
  WHERE dataset = 'test'
))
""")
result = q.to_dataframe().iloc[0]['mean_absolute_error']
result

77294.68408464122

In [33]:
#q9
q = bq.query(
"""
select weight 
from 
  ML.WEIGHTS(MODEL `p8.model`)
where
  processed_input = 'loan_term'
""")
q.to_dataframe().iloc[0]['weight']

381.8080365877182

In [49]:
#q10
q = bq.query(
"""
WITH predictions AS (
  SELECT
    income,
    predicted_loan_amount
  FROM
    ML.PREDICT(MODEL p8.model, (
      SELECT
        income,
        360 as loan_term
      FROM
        p8.hdma
    ))
)
SELECT
  SUM(CASE WHEN a.loan_amount > predictions.predicted_loan_amount THEN 1 ELSE 0 END) / COUNT(a.loan_amount) AS ratio_above_prediction
FROM
  predictions
JOIN p8.applications a ON a.income = predictions.income
""")
q.to_dataframe().iloc[0]['ratio_above_prediction']

0.6204781463640954